In [10]:
import copy
import numpy as np
import math

class HalmaAgent:
    def __init__(self, max_depth, evaluation_function):
        self.max_depth = max_depth
        self.evaluation_function = evaluation_function
        
    def minimax(self, game, max_depth, player, alpha, beta):
        best_move_max = None
        best_move_min = None
        if max_depth == 0:
            value = self.evaluation_function(game)
            return value, best_move_max, best_move_min
        
        if player == 2:
            value = float('-inf')
            possible_moves = game.moves_check(game.current_player)
            for move in possible_moves:
                evaluation,_,_ = self.minimax(move, max_depth - 1, 1, alpha, beta)
                if evaluation > value:
                    best_move_max = move
                value = max(value, evaluation)
                alpha = max(alpha, value)
                if beta <= alpha:
                    break
            return value, best_move_max, best_move_min
        
        elif player == 1:
            value = float('inf')
            possible_moves = game.moves_check(game.current_player)
            for move in possible_moves:
                evaluation,_,_ = self.minimax(move, max_depth - 1, 2, alpha, beta)
                if evaluation < value:
                    best_move_min = move
                value = min(value, evaluation)
                beta = min(beta, value)
                if beta <= alpha:
                    break
            return value, best_move_max, best_move_min

    def choose_move(self, game):
        alpha = float('-inf')
        beta = float('inf')
        value, best_move_max, best_move_min = self.minimax(game, self.max_depth, game.current_player, alpha, beta)
        return value, best_move_max, best_move_min


In [11]:
from copy import deepcopy
from copy import copy
import numpy as np


NUM_ROWS = 6
NUM_COLUMNS = 6
#TWO_P = [5, 5, 4, 3, 2]
TWO_P = [2,2]

class HalmaGame:
    def __init__(self, board_new=None):
        self.goal_board = np.zeros([NUM_ROWS, NUM_COLUMNS])
        for i in range(len(TWO_P)):
                self.goal_board[i, :TWO_P[i]] = 2
                self.goal_board[-i - 1, -TWO_P[i]:] = 1
        self.board = board_new
        if np.array_equal(board_new, None):
            self.board = np.zeros([NUM_ROWS, NUM_COLUMNS])
            for i in range(len(TWO_P)):
                self.board[i, :TWO_P[i]] = 1
                self.board[-i - 1, -TWO_P[i]:] = 2

        self.current_player = 1
        self.winner = -1

        self.child_boards = []
        self.parent = None
        self.parent_operator = None
        self.jump_flag = 0
        self.new_idx = None

    def __eq__(self, other):
        return np.array_equal(self.board, other.board)

    def __hash__(self):
        return hash(str([item for sublist in self.board for item in sublist]))

    # def check_winner(self):
    #    goal_player_1 = np.where(self.goal_board == 1)
    #    goal_player_2 = np.where(self.goal_board == 2)
    #    board_index_1 = np.where(self.board == 1)
    #    board_index_2 = np.where(self.board == 2)
    #    if np.array_equal(goal_player_1, board_index_1):
    #        return self.winner = 1
    #    elif np.array_equal(goal_player_2, board_index_2):
    #        return self.winner = 2
    #    else:
    #        return self.winner = -1

    def set_player(self, p):
        self.player = p
        
    def set_board(self, b):
        self.board = b
        
    def switch_player(self, state):
        if state.current_player == 1:
            return 2
        if state.current_player == 2:
            return 1

        
    def setSimpleMove(self, kid, parent_op, p, switch_p=0, jump_f = 0, newindex = None):
        kid.parent = p
        kid.parent_operator = parent_op
        if switch_p:
            kid.current_player = self.switch_player(switch_p)
        kid.jump_flag = jump_f
        kid.new_idx = newindex
        
      
    def moves_check(self, player):
        visited = set()
        visited.add(self)
        queue = [self]
        all_kids = set()
        jump_flag = 0
        pieces = np.array(np.where(self.board == player)).T

        #for idx in pieces:
        for i in range(len(pieces)):
            idx = deepcopy(pieces[i])
            # up
            if idx[0] - 1 >= 0:
                if not self.board[idx[0] - 1, idx[1]]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] - 1, idx[1]] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "up", self)
                    all_kids.add(child)

            # up_right
            if idx[0] - 1 >= 0 and idx[1] + 1 < NUM_COLUMNS:
                if not self.board[idx[0] - 1, idx[1] + 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] - 1, idx[1] + 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "up_right", self)
                    all_kids.add(child)

            # right
            if idx[1] + 1 < NUM_COLUMNS:
                if not self.board[idx[0], idx[1] + 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0], idx[1] + 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "right", self)
                    all_kids.add(child)

            # down_right
            if idx[0] + 1 < NUM_ROWS and idx[1] + 1 < NUM_COLUMNS:
                if not self.board[idx[0] + 1, idx[1] + 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] + 1, idx[1] + 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "down_right", self)
                    all_kids.add(child)

            # down
            if idx[0] + 1 < NUM_ROWS:
                if not self.board[idx[0] + 1, idx[1]]:
                    #child = deepcopy(self)
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] + 1, idx[1]] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "down", self)
                    all_kids.add(child)

            # down left
            if idx[0] + 1 < NUM_ROWS and idx[1] - 1 >= 0:
                if not self.board[idx[0] + 1, idx[1] - 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] + 1, idx[1] - 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "down_left", self)
                    all_kids.add(child)

            # left
            if idx[1] - 1 >= 0:
                if not self.board[idx[0], idx[1] - 1]:
                    #child = deepcopy(self)
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0], idx[1] - 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "left", self)
                    all_kids.add(child)

            # up_left
            if idx[0] - 1 >= 0 and idx[1] - 1 >= 0:
                if not self.board[idx[0] - 1, idx[1] - 1]:
                    new_board = deepcopy(self.board)
                    new_board[idx[0], idx[1]] = 0
                    new_board[idx[0] - 1, idx[1] - 1] = player
                    child = HalmaGame(new_board)
                    self.setSimpleMove(child, "up_left", self)
                    all_kids.add(child)

            while len(queue):
                node = queue.pop(0)
                if node.jump_flag:
                    idx = node.new_idx
                    
                if idx[0] - 2 >= 0:
                    if node.board[idx[0] - 1, idx[1]] and not node.board[idx[0] - 2, idx[1]]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] - 2, idx[1]] = player
                        child = HalmaGame(new_board)                      
                        if child not in visited:
                            self.setSimpleMove(child, "jump_up", node, jump_f = 1, newindex = [idx[0] - 2, idx[1]])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0


                # jump up right
                if idx[0] - 2 >= 0 and idx[1] + 2 < NUM_COLUMNS:
                    if node.board[idx[0] - 1, idx[1] + 1] and not node.board[idx[0] - 2, idx[1] + 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] - 2, idx[1] + 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_up_right", node, jump_f = 1, newindex = [idx[0] - 2, idx[1]+2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0
                            
                            
                # jump right
                if idx[1] + 2 < NUM_COLUMNS:
                    if node.board[idx[0], idx[1] + 1] and not node.board[idx[0], idx[1] + 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0], idx[1] + 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_right", node, jump_f = 1, newindex = [idx[0], idx[1] + 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0

                # jump down right
                if idx[0] + 2 < NUM_ROWS and idx[1] + 2 < NUM_COLUMNS:
                    if node.board[idx[0] + 1, idx[1] + 1] and not node.board[idx[0] + 2, idx[1] + 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] + 2, idx[1] + 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_down_right", node, jump_f = 1, newindex = [idx[0] + 2, idx[1] + 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0

                # jump down
                if idx[0] + 2 < NUM_ROWS:
                    if node.board[idx[0] + 1, idx[1]] and not node.board[idx[0] + 2, idx[1]]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] + 2, idx[1]] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_down", node, jump_f = 1, newindex = [idx[0] + 2, idx[1]])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0

                # jump down left
                if idx[0] + 2 < NUM_ROWS and idx[1] - 2 >= 0:
                    if node.board[idx[0] + 1, idx[1] - 1] and not node.board[idx[0] + 2, idx[1] - 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0] + 2, idx[1] - 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_down_left", node, jump_f = 1, newindex = [idx[0] + 2, idx[1] - 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0
                        
                # jump left
                if idx[1] - 2 >= 0:
                    if node.board[idx[0], idx[1] - 1] and not node.board[idx[0], idx[1] - 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0], idx[1] - 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_left", node, jump_f = 1, newindex = [idx[0], idx[1] - 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0
                        
                # jump up left
                if idx[1] - 2 >= 0 and idx[0] -2 >= 0:
                    if node.board[idx[0]-1, idx[1] - 1] and not node.board[idx[0]-2, idx[1] - 2]:
                        new_board = deepcopy(node.board)
                        new_board[idx[0], idx[1]] = 0
                        new_board[idx[0]-2, idx[1] - 2] = player
                        child = HalmaGame(new_board)
                        if child not in visited:
                            self.setSimpleMove(child, "jump_up_left", node, jump_f = 1, newindex = [idx[0]-2, idx[1] - 2])
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                        else:
                            node.current_player = self.switch_player(node)
                            node.jump_flag = 0
                        
            queue=[self]
        self.child_boards = list(all_kids)
        return tuple(self.child_boards)

In [12]:
# Define evaluation functions

def evaluation_function_1(game):
    board_goal = game.goal_board
    board_state = game.board
    indexes_target_2 = np.where(board_goal == 2)
    indexes_in_2 = np.where(board_goal == 1)
    indexes_state_2 = np.where(board_state == 2)
    indexes_state_1 = np.where(board_state == 1)
    index_list_inital_1 = []
    index_list_target_1 = []
    index_list_inital_2 = []
    index_list_target_2 = []
    index_list_2 = []
    index_list_1 = []
    
    
    for i in range(len(indexes_in_2[0])):
        index_in_2 = [indexes_in_2[0][i], indexes_in_2[1][i]]
        index_list_inital_2.append(index_in_2)
        index_tg_2 = [indexes_target_2[0][i], indexes_target_2[1][i]]
        index_list_target_2.append(index_tg_2)
        index_2 = [indexes_state_2[0][i], indexes_state_2[1][i]]
        index_list_2.append(index_2)
        index_1 = [indexes_state_1[0][i], indexes_state_1[1][i]]
        index_list_1.append(index_1)

    index_list_inital_1 = index_list_target_2
    index_list_target_1 = index_list_inital_2

    min_dist_1 = []
    min_dist_2 = []
    for i in range(len(index_list_inital_2)):
        dist_it_1 = []
        dist_it_2 = []
        for j in range(len(index_list_target_2)):
            if index_list_1[i] not in index_list_target_1:
                dist_it_1.append(math.dist(index_list_1[i], index_list_target_1[j]))
                dist_it_2.append(math.dist(index_list_2[i], index_list_target_2[j]))
        min_dist_1.append(max(dist_it_1))
        min_dist_2.append(max(dist_it_2))

    distances_1 = round(sum(min_dist_1), 5)
    distances_2 = round(sum(min_dist_2), 5)
    
    diff = distances_1 - distances_2

    return diff

# Create two agents with different levels of difficulty

game = HalmaGame()
#x = game.moves_check(game, game.current_player)

agent1 = HalmaAgent(max_depth=2, evaluation_function=evaluation_function_1)
#agent2 = HalmaAgent(max_depth=4, evaluation_function=evaluate2)
#print(evaluation_function_1(x[0]))

# Run the game until it's over
value, best_move_max, best_move_min = agent1.choose_move(game)
print('value', value)
#print('best move max', best_move_max.board)
print(best_move_min.board)


value 0.0
[[1. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 2.]
 [0. 0. 0. 0. 2. 2.]]


In [13]:
best_move_min.parent_operator

'down_right'

In [14]:
for i in g1

SyntaxError: invalid syntax (2202968644.py, line 1)

In [15]:
g1 = HalmaGame()
print(len(g1.moves_check(1)))
for i in g1.child_boards:
    print(i.goal_board)

14
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]
[[2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0

In [16]:
g1.child_boards[0].parent.board

array([[1., 1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 2., 2.],
       [0., 0., 0., 0., 2., 2.]])

In [18]:
len(g1.child_boards)


14

In [155]:
l = [1,2,3,4]
x = l[2]

In [156]:
id(l[2])

4403526000

In [157]:
id(x)

4403526000

In [158]:
x = 6

In [159]:
l

[1, 2, 3, 4]

In [160]:
id(x)

4403526096